## Instacart Market Basket Analysis
The objective is to predict which customer will repurchase a product in his current order, based on a anonymized dataset containing 3,000,000 orders.

Each user then has about 4 - 100 orders.

A user's reorder depends on User, the products that he purchases and the history of his purchases 

 

## Extraction and Loading 

Lets take  look at a  few datasets provided

In [34]:
% matplotlib inline

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt


Read in the files 

In [35]:
order_products_train_df = pd.read_csv("../../input/order_products__train.csv")
order_products_prior_df = pd.read_csv("../../input/order_products__prior.csv")
orders_df = pd.read_csv("../../input/orders.csv")
products_df = pd.read_csv("../../input/products.csv")
aisles_df = pd.read_csv("../../input/aisles.csv")
departments_df = pd.read_csv("../../input/departments.csv")

## Data Exploration and  Data Wrangling

### Explore  Products data frame

In [36]:
products_df.head()

,product_id,product_name,aisle_id,department_id
0,1,Chocolate Sandwich Cookies,61,19
1,2,All-Seasons Salt,104,13
2,3,Robust Golden Unsweetened Oolong Tea,94,7
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1
4,5,Green Chile Anytime Sauce,5,13


In [37]:
(rows, cols ) = products_df.shape
print("There are {} procucts".format(rows))

There are 49688 procucts


In [38]:
products_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49688 entries, 0 to 49687
Data columns (total 4 columns):
product_id       49688 non-null int64
product_name     49688 non-null object
aisle_id         49688 non-null int64
department_id    49688 non-null int64
dtypes: int64(3), object(1)
memory usage: 1.5+ MB


####   RESULT Luckily there are __no missing values__ in product_df : it is clean

### Next, Explore Aisles data-frame

In [39]:
aisles_df.head()

,aisle_id,aisle
0,1,prepared soups salads
1,2,specialty cheeses
2,3,energy granola bars
3,4,instant foods
4,5,marinades meat preparation


In [40]:
aisles_df.shape
(rows, cols) = aisles_df.shape
print("There are {} aisles in the aisles_df".format(rows))

There are 134 aisles in the aisles_df


In [41]:
aisles_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 134 entries, 0 to 133
Data columns (total 2 columns):
aisle_id    134 non-null int64
aisle       134 non-null object
dtypes: int64(1), object(1)
memory usage: 2.2+ KB


#### RESULT : aisles_df is also clean : there are no missing values

###  Department data-frame 

In [42]:
departments_df.head()

,department_id,department
0,1,frozen
1,2,other
2,3,bakery
3,4,produce
4,5,alcohol


In [43]:
departments_df.shape
(rows, cols) = departments_df.shape
print("There are {} departments".format(rows))

There are 21 departments


In [44]:
departments_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21 entries, 0 to 20
Data columns (total 2 columns):
department_id    21 non-null int64
department       21 non-null object
dtypes: int64(1), object(1)
memory usage: 416.0+ bytes


####  RESULT : departments_df is also clean

#### Orders

In [57]:
orders_df.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,prior,1,2,8,19.0
1,2398795,1,prior,2,3,7,15.0
2,473747,1,prior,3,3,12,21.0
3,2254736,1,prior,4,4,7,29.0
4,431534,1,prior,5,4,15,28.0


In [58]:
(rows, cols) = orders_df.shape

In [59]:
print(" There are {} unique orders in the orders set".format(rows))

 There are 3421083 unique orders in the orders set


In [60]:
orders_df.groupby('eval_set')['order_id'].count()

eval_set
prior    3214874
test       75000
train     131209
Name: order_id, dtype: int64

In [61]:
orders_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3421083 entries, 0 to 3421082
Data columns (total 7 columns):
order_id                  int64
user_id                   int64
eval_set                  object
order_number              int64
order_dow                 int64
order_hour_of_day         int64
days_since_prior_order    float64
dtypes: float64(1), int64(5), object(1)
memory usage: 182.7+ MB


There seem to be missing values

In [62]:
orders_dropna_df = orders_df.dropna()
orders_dropna_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3421083 entries, 0 to 3421082
Data columns (total 7 columns):
order_id                  int64
user_id                   int64
eval_set                  object
order_number              int64
order_dow                 int64
order_hour_of_day         int64
days_since_prior_order    float64
dtypes: float64(1), int64(5), object(1)
memory usage: 208.8+ MB


In [63]:
print("There are {} missing values".format(len(orders_df) - len(orders_dropna_df)))

There are 0 missing values


In [64]:
print("This represents {}% of total orders ".format(((len(orders_df) -len(orders_dropna_df))* 100/ len(orders_df))))

This represents 0.0% of total orders 


In [65]:
orders_df_na = (orders_df.isnull())
orders_df_na.head()
for col_name in orders_df_na.columns.values:
    num_of_nan = len(orders_df_na[orders_df_na[col_name] == True])
    print(" There are {0} missing values in  column {1} in the orders_df data_frame".format( num_of_nan, col_name))

 There are 0 missing values in  column order_id in the orders_df data_frame
 There are 0 missing values in  column user_id in the orders_df data_frame
 There are 0 missing values in  column eval_set in the orders_df data_frame
 There are 0 missing values in  column order_number in the orders_df data_frame
 There are 0 missing values in  column order_dow in the orders_df data_frame
 There are 0 missing values in  column order_hour_of_day in the orders_df data_frame
 There are 0 missing values in  column days_since_prior_order in the orders_df data_frame


lets convert this from a data frame to a dictionary

In [66]:
## Lets calculate the 
orders_userid_df = orders_df.groupby('user_id')['days_since_prior_order'].mean()
orders_userid_df.head()
#orders_userid_df.index
days_since_dict = dict()
days_since_dict = orders_userid_df.to_dict()

In [67]:

#dict_variable = {key:value for (key,value) in dictonary.items()}
#

#mean_days_since_order_dict = {k:list(orders_userid_df.loc[k:1]) for k in orders_userid_df.index}


 

In [68]:
#orders_df['days_since_prior_order'].apply(lambda x: mean_days_since_order_dict[user_id] if pd.notnull[x] else x)
#df.order_prior = df3.order_prior.fillna(df.user_id.apply(lambda x: dic_map.get(x)))

orders_df['days_since_prior_order'] = orders_df.apply(lambda row: days_since_dict[row['user_id']] 
                                                            if (np.isnan(row['days_since_prior_order']) or pd.isnull(row['days_since_prior_order'])) 
                                                              else row['days_since_prior_order'], axis=1)




In [69]:
orders_df.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,prior,1,2,8,19.0
1,2398795,1,prior,2,3,7,15.0
2,473747,1,prior,3,3,12,21.0
3,2254736,1,prior,4,4,7,29.0
4,431534,1,prior,5,4,15,28.0


In [73]:
orders_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3421083 entries, 0 to 3421082
Data columns (total 7 columns):
order_id                  int64
user_id                   int64
eval_set                  object
order_number              int64
order_dow                 int64
order_hour_of_day         int64
days_since_prior_order    float64
dtypes: float64(1), int64(5), object(1)
memory usage: 182.7+ MB


Now we look at each column and ensure that there is no missing values in order_df

In [74]:
orders_df_na = (orders_df.isnull())
orders_df_na.head()
for col_name in orders_df_na.columns.values:
    num_of_nan = len(orders_df_na[orders_df_na[col_name] == True])
    print(" There are {0} missing values in  column {1} in the orders_df data_frame".format( num_of_nan, col_name))

 There are 0 missing values in  column order_id in the orders_df data_frame
 There are 0 missing values in  column user_id in the orders_df data_frame
 There are 0 missing values in  column eval_set in the orders_df data_frame
 There are 0 missing values in  column order_number in the orders_df data_frame
 There are 0 missing values in  column order_dow in the orders_df data_frame
 There are 0 missing values in  column order_hour_of_day in the orders_df data_frame
 There are 0 missing values in  column days_since_prior_order in the orders_df data_frame


In [75]:
orders_df.to_csv("../../input/orders_cleaned.csv",  index=False)

In [76]:
orders_cleaned_df = pd.read_csv("../../input/orders_cleaned.csv")

In [77]:
orders_cleaned_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3421083 entries, 0 to 3421082
Data columns (total 7 columns):
order_id                  int64
user_id                   int64
eval_set                  object
order_number              int64
order_dow                 int64
order_hour_of_day         int64
days_since_prior_order    float64
dtypes: float64(1), int64(5), object(1)
memory usage: 182.7+ MB


In [78]:
orders_cleaned_df.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,prior,1,2,8,19.0
1,2398795,1,prior,2,3,7,15.0
2,473747,1,prior,3,3,12,21.0
3,2254736,1,prior,4,4,7,29.0
4,431534,1,prior,5,4,15,28.0


A Read back of the saved csv file reveals that everthing is intact

#### Training set orders

In [79]:
order_products_train_df.head()

,order_id,product_id,add_to_cart_order,reordered
0,1,49302,1,1
1,1,11109,2,1
2,1,10246,3,0
3,1,49683,4,0
4,1,43633,5,1


As seen in the table above, for each unique order_id, we have product, the order in which it was ordered and whether this was a product thatwas re-odered.

In [80]:
order_products_train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1384617 entries, 0 to 1384616
Data columns (total 4 columns):
order_id             1384617 non-null int64
product_id           1384617 non-null int64
add_to_cart_order    1384617 non-null int64
reordered            1384617 non-null int64
dtypes: int64(4)
memory usage: 42.3 MB


order_products_train_df is clean and reqruires no further cleaning

### Order Products prior

In [82]:
order_products_prior_df.head()

,order_id,product_id,add_to_cart_order,reordered
0,2,33120,1,1
1,2,28985,2,1
2,2,9327,3,0
3,2,45918,4,1
4,2,30035,5,0


In [83]:
order_products_prior_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32434489 entries, 0 to 32434488
Data columns (total 4 columns):
order_id             int64
product_id           int64
add_to_cart_order    int64
reordered            int64
dtypes: int64(4)
memory usage: 989.8 MB


In [86]:
order_products_prior_na = (order_products_prior_df.isnull())
order_products_prior_na.head()
for col_name in order_products_prior_na.columns.values:
    num_of_nan = len(order_products_prior_na[order_products_prior_na[col_name] == True])
    print(" There are {0} missing values in  column {1} in the orders_df data_frame".format( num_of_nan, col_name))

 There are 0 missing values in  column order_id in the orders_df data_frame
 There are 0 missing values in  column product_id in the orders_df data_frame
 There are 0 missing values in  column add_to_cart_order in the orders_df data_frame
 There are 0 missing values in  column reordered in the orders_df data_frame


## SUMMARY
In summary,  there was only one data-set that required cleaning. The orders_df which is a history of the all the orders had  many NaNs in the day's since prior order. Upon closer look, it appeared that the very first order would not have known value for 'days since prior value'. In theory we could drop this entry. But in the academic interest of clening the data, for each user, the mean 'days_since_prior_order' was used for imputation of the value.

The following dataframes are declared CLEAN

order_products_train_df  
order_products_prior_df   
products_df  
aisles_df  
departments_df  

The orders_df data frame  has been cleaned with imputed values and written into a new csv file "../../input/orders_cleaned.csv"

For further ananlysis, the cleaned values will be used.